In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale, LabelBinarizer
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE, f_regression
#from minepy import MINE
from mlxtend.regressor import StackingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import gc
color = sns.color_palette()
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
import tensorflow as tf

/home/dope/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/dope/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
sample=pd.read_csv('/home/dope/data_set/medicine/f_sample_20180204.csv')
train=pd.read_csv('/home/dope/data_set/medicine/f_train_20180204.csv',encoding='gbk')
test=pd.read_csv('/home/dope/data_set/medicine/f_test_a_20180204.csv',encoding='gbk')

In [3]:
train.columns.values

array(['id', 'SNP1', 'SNP2', 'SNP3', 'SNP4', 'SNP5', 'SNP6', 'SNP7',
       'SNP8', 'SNP9', 'SNP10', 'SNP11', 'SNP12', 'SNP13', 'SNP14',
       'SNP15', 'SNP16', 'SNP17', 'SNP18', 'SNP19', 'SNP20', 'SNP21',
       'SNP22', 'SNP23', 'RBP4', '年龄', '孕次', '产次', '身高', '孕前体重', 'BMI分类',
       '孕前BMI', '收缩压', '舒张压', '分娩时', '糖筛孕周', 'VAR00007', 'wbc', 'ALT',
       'AST', 'Cr', 'BUN', 'CHO', 'TG', 'HDLC', 'LDLC', 'ApoA1', 'ApoB',
       'Lpa', 'hsCRP', 'SNP24', 'SNP25', 'SNP26', 'SNP27', 'SNP28',
       'SNP29', 'SNP30', 'SNP31', 'SNP32', 'SNP33', 'SNP34', 'SNP35',
       'SNP36', 'SNP37', 'SNP38', 'DM家族史', 'SNP39', 'SNP40', 'SNP41',
       'SNP42', 'SNP43', 'SNP44', 'SNP45', 'SNP46', 'SNP47', 'SNP48',
       'SNP49', 'SNP50', 'SNP51', 'SNP52', 'SNP53', 'SNP54', 'SNP55',
       'ACEID', 'label'], dtype=object)

# 定义处理数据函数

In [4]:
def my_dropna(df):
    des=df.describe().ix[0]
    index=0
    for col in df.columns.values:
        if des[index]/len(df[col]) <= 0.81:
            df.drop(col,axis=1,inplace=True)
        index=index+1

In [5]:
def my_fillna(df):
    df.fillna(df.mean(),inplace=True)

In [6]:
def my_norm(df):
    for col in df.columns.values:
        if df[col].mean() > 3 :
            df[col] = df[col] / (df[col].mean() / 3)
            #for index in range(len(df[col])):
            #   df[col].ix[index] = df[col].ix[index] / ( df[col].mean() / 4 )

In [7]:
def norm(df):
    for j in [col for col in df]:
        t_max=df[j].max()
        t_min=df[j].min()
        down = 1 if (t_max-t_min == 0) else (t_max-t_min)
        df[j]=(df[j]-t_min) / down * 255

# Data Pre

In [8]:
my_dropna(train)
my_fillna(train)
my_norm(train)

/home/dope/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [9]:
label_train=train['label']
train.drop('label',axis=1,inplace=True)
train.drop('id',axis=1,inplace=True)

In [10]:
y_train=[]
for index , value in enumerate(label_train):
    y_train.append([1,0]) if value == 1 else y_train.append([0,1])

In [11]:
train

,SNP1,SNP2,SNP3,SNP4,SNP5,SNP6,SNP7,SNP8,SNP9,SNP10,...,SNP44,SNP45,SNP46,SNP47,SNP48,SNP49,SNP50,SNP51,SNP52,SNP53
0,2,1,2.796414,2.0,3.000000,1.0,3.0,3.0,2.0,2.0,...,2.000000,2.000000,1.000000,2.000000,2.0,2.000000,1.000000,2.000000,1.0,1.000000
1,3,2,3.000000,2.0,2.000000,2.0,3.0,3.0,2.0,2.0,...,1.000000,1.000000,2.000000,2.000000,1.0,3.000000,1.000000,1.000000,2.0,1.000000
2,1,1,3.000000,1.0,3.000000,2.0,3.0,3.0,2.0,2.0,...,1.000000,3.000000,2.000000,2.000000,3.0,3.000000,2.000000,2.000000,1.0,1.000000
3,1,3,3.000000,1.0,2.098396,1.0,3.0,1.0,1.0,1.0,...,2.000000,1.000000,2.000000,2.000000,1.0,3.000000,1.000000,2.000000,1.0,1.000000
4,1,2,3.000000,2.0,2.000000,1.0,2.0,2.0,1.0,2.0,...,1.000000,2.000000,1.000000,1.000000,3.0,1.000000,2.000000,2.000000,1.0,1.000000
5,2,2,3.000000,2.0,1.000000,2.0,3.0,2.0,1.0,2.0,...,1.000000,2.000000,1.000000,2.000000,2.0,2.000000,1.000000,1.000000,2.0,1.000000
6,2,2,3.000000,1.0,3.000000,1.0,3.0,3.0,2.0,2.0,...,1.000000,2.000000,1.000000,3.000000,3.0,2.000000,1.000000,2.000000,1.0,1.000000
7,3,2,3.000000,2.0,2.000000,2.0,3.0,2.0,1.0,2.0,...,1.000000,2.000000,3.000000,1.905561,1.0,3.000000,1.000000,2.000000,1.0,1.000000
8,2,2,3.000000,1.0,1.000000,1.0,3.0,3.0,2.0,2.0,...,1.000000,1.000000,1.000000,1.000000,1.0,2.000000,1.000000,1.000000,2.0,1.000000
9,1,2,3.000000,2.0,2.000000,2.0,2.0,3.0,2.0,2.0,...,2.000000,3.000000,3.000000,2.000000,3.0,3.000000,1.000000,1.000000,2.0,1.000000


for col in train.columns.values:
    print(col + ":" + str(train[col].mean()))

In [12]:
np.array(train.ix[0]).reshape([8,8,1])

/home/dope/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


array([[[  2.        ],
        [  1.        ],
        [  2.7964135 ],
        [  2.        ],
        [  3.        ],
        [  1.        ],
        [  3.        ],
        [  3.        ]],

       [[  2.        ],
        [  2.        ],
        [  2.        ],
        [  1.        ],
        [  1.        ],
        [  1.        ],
        [  2.        ],
        [  3.        ]],

       [[  2.        ],
        [  3.        ],
        [  3.        ],
        [  1.        ],
        [  2.44911761],
        [  1.57484647],
        [  2.42600192],
        [  6.35030082]],

       [[  2.52458741],
        [  2.21      ],
        [  2.43121662],
        [  2.4       ],
        [  1.71      ],
        [  2.51853144],
        [ 21.7       ],
        [  9.7       ]],

       [[  0.1684122 ],
        [  1.76905984],
        [  1.15862808],
        [  1.        ],
        [  1.        ],
        [  2.        ],
        [  1.        ],
        [  2.        ]],

       [[  1.        ],
      

# CNN

In [13]:
INPUT_WIDTH=8
CONV_WIDTH=3
TRAINNING_STEP=1000
IN_CHANNEL_SIZE=1
OUT_CHANNEL_SIZE=32
OUTPUT_SIZE=2

LEARNING_RATE_BASE = 0.005
LEARNING_RATE_DECAY = 0.99

In [14]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def get_train_data(data,times):
    return_data=[]
    for i in range(10):
        return_data.append(np.array(train.ix[times*10+i]).reshape([8,8,1]))
    return return_data
def get_train_y(label,times):
    return_y=[]
    for i in range(10):
        return_y.append(y_train[times*10+i])
    return return_y

In [15]:
with tf.variable_scope("cnn",reuse=tf.AUTO_REUSE):
    train_X = tf.placeholder(tf.float32,[None,INPUT_WIDTH,INPUT_WIDTH,1])
    train_y = tf.placeholder(tf.float32,[None,OUTPUT_SIZE])
    keep_prob = tf.placeholder(tf.float32)
    global_step = tf.Variable(0, trainable=False)
    # conv1 layer
    #train_X = tf.reshape(train_X,[-1,INPUT_WIDTH,INPUT_WIDTH,1])
    W_conv1 = weight_variable([INPUT_WIDTH,INPUT_WIDTH,IN_CHANNEL_SIZE,OUT_CHANNEL_SIZE])
    b_conv1 = bias_variable([OUT_CHANNEL_SIZE])
    h_conv1 = tf.nn.relu(conv2d(train_X,W_conv1)+b_conv1) #output size is 8x8x2
    h_pool1 = max_pool_2x2(h_conv1) #output size is 4x4xOUT_CHANNEL_SIZE
    
    #func1 layer
    W_fc1 = weight_variable([4*4*OUT_CHANNEL_SIZE,6])
    b_fc1 = bias_variable([6])
    h_pool1_flat = tf.reshape(h_pool1,[-1,4*4*OUT_CHANNEL_SIZE])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    #func2 layer
    W_fc2 = weight_variable([6,2])
    b_fc2 = bias_variable([2])
    prediction = tf.nn.softmax(tf.matmul(h_fc1,W_fc2) + b_fc2)
    
    #ops
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(train_y * tf.log(prediction),reduction_indices=[1]))
    #loss_op = tf.reduce_mean(tf.square(train_y - prediction))
    
    #other param
    learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            200, LEARNING_RATE_DECAY,
            staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cross_entropy,global_step=global_step)
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

In [20]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        _data=get_train_data(train,i)
        _label=get_train_y(y_train,i)
        sess.run(train_op,feed_dict={train_X:_data , train_y:_label , keep_prob:0.3})
    #sess.run(train_op,feed_dict={train_X: [np.array(train.ix[0]).reshape([8,8,1]) , np.array(train.ix[1]).reshape([8,8,1])] , train_y:[y_train[0],y_train[1]] , keep_prob:0.3})
    print(sess.run(prediction,feed_dict={train_X:_data , train_y:_label }))
    #print(W_conv1.eval())

[[ 0.75036842  0.24963155]
 [ 0.87654406  0.12345597]
 [ 0.25992146  0.74007851]
 [ 0.71237636  0.28762355]
 [ 0.67443764  0.32556242]
 [ 0.64958066  0.35041931]
 [ 0.7244423   0.2755577 ]
 [ 0.77817845  0.22182153]
 [ 0.7344768   0.26552317]
 [ 0.46414545  0.53585458]]


In [17]:
y_train

[[1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 